In [1]:
#r "nuget:Voice100"
#r "nuget:Voice100.Native"
#r "nuget:Microsoft.ML.OnnxRuntime"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.OnnxRuntime, 1.11.0 Voice100, 1.2.1

In [2]:
using System.Net.Http;
using Voice100;

In [3]:
string cacheDirectoryPath = "./cache";
bool useMultiTaskTTS = true;

In [4]:
async Task<SpeechSynthesizer> BuildSpeechSynthesizerAsync(bool useMultiTaskTTS)
{
    string alignModelPath;
    string audioModelPath;
    using (var httpClient = new HttpClient())
    {
        var downloader = new ModelDownloader(httpClient, cacheDirectoryPath);
        alignModelPath = await downloader.MayDownloadAsync(
            "ttsalign_en_conv_base-20210808.onnx",
            "https://github.com/kaiidams/voice100-runtime/releases/download/v0.1/ttsalign_en_conv_base-20210808.onnx",
            "D87B80B2C9CC96AC7A4C89C979C62FA3C18BACB381C3C1A3F624A33496DD1FC8");
        if (useMultiTaskTTS)
        {
            audioModelPath = await downloader.MayDownloadAsync(
                "ttsaudio_en_mt_conv_base-20220316.onnx",
                "https://github.com/kaiidams/voice100-runtime/releases/download/v1.2.0/ttsaudio_en_mt_conv_base-20220316.onnx",
                "5d0f426509bb662deab3ca9cf964f68dbaf2a30b55e653205c98eaad63978468");
        }
        else
        {
            audioModelPath = await downloader.MayDownloadAsync(
                "ttsaudio_en_conv_base-20220107.onnx",
                "https://github.com/kaiidams/voice100-runtime/releases/download/v1.0.1/ttsaudio_en_conv_base-20220107.onnx",
                "A20FEC366D1A4856006BBF7CFAC7D989EF02B0C1AF676C0B5E6F318751325A2F");
        }
    }
    return new SpeechSynthesizer(alignModelPath, audioModelPath, useMultiTaskTTS);
}

In [5]:
var synthesizer = await BuildSpeechSynthesizerAsync(useMultiTaskTTS);

Using cached `ttsalign_en_conv_base-20210808.onnx'.
Using cached `ttsaudio_en_mt_conv_base-20220316.onnx'.


In [6]:
synthesizer.Speak("hello");

Unhandled exception: System.DllNotFoundException: Unable to load shared library 'voice100_native' or one of its dependencies. In order to help diagnose loading problems, consider setting the LD_DEBUG environment variable: libvoice100_native: cannot open shared object file: No such file or directory
   at Voice100.WORLDVocoder.Voice100Sharp_VocoderDecode(Single[] f0, Single[] logspc, Single[] codedap, Int32 f0_length, Int32 fft_size, Double frame_period, Int32 fs, Single log_offset, Int16[] y, Int32 y_length)
   at Voice100.WORLDVocoder.Decode(Single[] f0, Single[] logspc, Single[] codeap)
   at Voice100.SpeechSynthesizer.Predict(Int64[] aligned)
   at Voice100.SpeechSynthesizer.Speak(String text, Int16[]& audio)
   at Voice100.SpeechSynthesizer.Speak(String text)
   at Submission#7.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, F